### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
# parallel
from functools import partial
from pathlib import Path

In [4]:
from data_sources import *
from midi_data import *

In [5]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version
orig_path = version_path/'midi_sources'
metapath = version_path/'metadata'
all_csv = metapath/'midi_sources.csv'
metapath.mkdir(parents=True, exist_ok=True)

In [6]:
def create_paths(dirname):
    "Standardize midi_source paths"
    dir_path = orig_path/dirname
    json_path = metapath/f'{dirname}_metadata.json'
    csv_path = metapath/f'{dirname}_metadata.csv'
    return dir_path, json_path, csv_path

In [7]:
sources = ['hooktheory', 'freemidi', 'midiworld', 'ecomp', 'cprato', 'classic_piano', 'wikifonia']

In [8]:
version_path.relative_to(data_path)

PosixPath('v5')

In [9]:
def relative_path(filepath):
    return str(Path(filepath).relative_to(version_path))

### Remove corrupted file - this causes deadlock with music21 processing

In [10]:
corrupted_files = [
    'midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid',
    'ecomp/2004/MORET02.mid',
    'ecomp/2006/Mordvinov9.MID',
    'ecomp/2006/Na06.MID',
    'ecomp/2008/Cui01.MID',
    'ecomp/2008/Cui02.MID',
    'ecomp/2008/Cui03.MID',
    'ecomp/2008/Cui04.MID',
    'ecomp/2008/Cui05.MID',
    'ecomp/2008/Cui06.MID',
    'ecomp/2008/Cui07.MID',
    'ecomp/2008/Cui08.MID',
    'ecomp/2008/Tan01.MID',
    'ecomp/2008/Tan02.MID',
    'ecomp/2008/Tan03.MID',
    'ecomp/2018/KaszoS14.MID',
    'midiworld/named_midi/Rob_Zombie_-_Demonoid_Phenomenon.mid',
    'midiworld/named_midi/Rob_Zombie_-_Demonoid_Phenomenon.mxl',
]
for f in corrupted_files:
    fp = orig_path/f
    if fp.exists(): fp.unlink()

### Hooktheory

In [11]:
ht_path, ht_json, ht_csv = create_paths('hooktheory')
ht_song_list = metapath/'hooktheory_key2info.json'

In [12]:
def song_key(s): return '_'.join(s.parts[-3:-1])

In [13]:
ht_key2info = load_json(ht_song_list)

if ht_key2info is None:
    song_info = list((ht_path/'xml').glob('*/*/*/*.json'))
    ht_key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(ht_key2info, ht_song_list)
len(ht_key2info)

11873

In [14]:
ht_midi_list = list((ht_path/'pianoroll').glob('*/*/*/*_key.mid')); 
len(ht_midi_list)

19876

In [15]:
def get_ht_jsonfile(midi_file): # using json instead of midi for metadata
    return str(midi_file.with_suffix('.json')).replace('pianoroll', 'event').replace('_key', '_symbol_key')

In [16]:
def get_hooktheory_attr(fp):
    song_info = ht_key2info[song_key(fp)]
    song_json = json.load(open(get_ht_jsonfile(fp), 'r'))
    metadata = song_json['metadata']
    artist = fp.parts[-3]
    title = fp.parts[-2]
    section = fp.name.split('_')[0]
    
    ht_key = metadata['key']
    ht_mode = metadata['mode']
    if ht_mode is None: ht_mode = 'major'
    ht_offset = keyc_offset(ht_key, ht_mode)
    
    # convert stream here
    return {
        'artist': artist,
        'title': title,
        'midi': relative_path(fp),
        'section': section,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'source': 'hooktheory',
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_offset': ht_offset,
        'ht_time_signature': metadata['beats_in_measure']
    }

In [17]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000]); hook_out

In [18]:
ht_metadata = parse_midi_dir(ht_midi_list, ht_json, base_path=version_path, 
                             meta_func=get_hooktheory_attr)

Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/w/weezer/beverly-hills/intro-and-verse_key.mid list index out of range


Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/y/ylvis/the-fox---what-does-the-fox-say/verse-and-pre-chorus_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/j/jay-z/so-ambitious-feat-pharrel-williams/intro_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/j/johnny-cash/folsom-prison-blues/verse_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/j/jack-johnson/do-you-remember/intro-and-verse_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/j/justin-moore/if-heaven-wasnt-so-far-awat/intro-and-verse_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/j/john-cage/433/solo_key.mid list index out of ran

Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/q/queen/bohemian-rhapsody/pre-outro_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/a/adventure-club/wonder/outro_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/a/adventure-club/wonder/bridge_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/a/antonio-vivaldi/the-four-seasons-concerto-no-4-winter/intro_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/a/adelina-tahiri/heart-on-fire/intro_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/a/antonim/melancholy-soldier/bridge_key.mid list index out of range
Midi Exeption: data/midi/v5/midi_sources/hooktheory/pianoroll/a/asf/asdfas/intro-and-verse_key.mid list index out of range


#### Save song_list

In [19]:
ht_metadata = load_json(ht_json)
arr2csv(ht_metadata.values(), ht_csv)
df = pd.read_csv(ht_csv); df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,bpm,midi,inferred_key,inferred_offset,genres,section,parts,ht_time_signature,ht_bpm,midi_title
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,85.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,0.0,NaN,intro,"intro,chorus",3,85,yu-gi-oh
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,128.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,0.0,NaN,chorus,"intro,chorus",4,128,yu-gi-oh3
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,96.0,midi_sources/hooktheory/pianoroll/w/what-a-day...,A minor,0.0,Jazz,chorus,chorus,4,96,kiefer
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,152.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,D minor,-5.0,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4,152,Senbonzakura
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,108.0,midi_sources/hooktheory/pianoroll/w/wham/last-...,C# major,-1.0,Holiday,intro,"intro,verse,chorus",4,108,Last Christmas Intro


In [20]:
df.shape

(19876, 21)

## FreeMidi

In [21]:
fm_path, _, fm_csv = create_paths('freemidi')
fm_dance_path = metapath/f'freemidi_dance_metadata.json'
fm_pop_path = metapath/f'freemidi_pop_metadata.json'
list(fm_path.glob('*'))

[PosixPath('data/midi/v5/midi_sources/freemidi/genre-disco'),
 PosixPath('data/midi/v5/midi_sources/freemidi/genre-pop'),
 PosixPath('data/midi/v5/midi_sources/freemidi/genre-dance-eletric'),
 PosixPath('data/midi/v5/midi_sources/freemidi/genre-punk'),
 PosixPath('data/midi/v5/midi_sources/freemidi/genre-hip-hop-rap'),
 PosixPath('data/midi/v5/midi_sources/freemidi/genre-rock')]

In [22]:
def parse_freemidi_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': genres,
        'source': source
    }

In [23]:
d_parse_func = partial(parse_freemidi_songs, genres='dance', source='freemidi')
dir_path = fm_path/'genre-dance-eletric'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_dance_list = parse_midi_dir(file_list, fm_dance_path,
                               base_path=version_path, meta_func=d_parse_func)

Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-dance-eletric/Gloria Estefan - I'm Not Givin' You Up.mid badly formated midi bytes, got: b'RIFFB\x8c\x00\x00RMIDdata~\x8b\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-dance-eletric/Atomic Kitten - Whole Again.mid badly formated midi bytes, got: b'RIFF\x08K\x00\x00RMIDdata{J\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-dance-eletric/Apollo 440 - Lost In Space.mid index out of range
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-dance-eletric/Tatu - Ya Tvoy Vrag (I'm Your Enemy).mid badly formated midi bytes, got: b'RIFF,\xa3\x00\x00RMIDdata\xc6\xa2\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-dance-eletric/Gloria Estefan - Get On Your Feet.mid badly formated midi bytes, got: b'RIFF\x92\xa8\x01\x00RMIDdata\xd8\xa7\x01\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-dance-eletric/Gloria Estefan - Mi Tierra.mid badly formated midi bytes, got: b'RIFFz\r\x

In [24]:
p_parse_func = partial(parse_freemidi_songs, genres='pop', source='freemidi')
dir_path = fm_path/'genre-pop'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_pop_list = parse_midi_dir(file_list, fm_pop_path,
                             base_path=version_path, meta_func=p_parse_func)

Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Cyndi Lauper - Who Let In The Rain.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Cyndi Lauper - Whats Going On.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/5th Dimension - One Less Bell To Answere.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Cyndi Lauper - The World Is Stone.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Janet Jackson - Escapade.mid badly formated midi bytes, got: b'error with file'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Cyndi Lauper - Change Of Heart.mid badly formated midi bytes, got: b'0&\

Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Westlife - Fool Again.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Modern Talking - Megamix 2000.mid badly formated midi bytes, got: b'RIFFp\x91\x02\x00RMIDdata\xd1\x90\x02\x00'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Donna Summer - Bad Girls.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Britney Spears - I Wanna Go.mid index out of range
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Gloria Estefan - Abriendo Puertas.mid badly formated midi bytes, got: b'error with file'
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Four Seasons - I've Got You Under My Skin.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/freemidi/genre-pop/Wings - Band On The Run.mid badly formed midi string: missing leading MTrk
Midi Exeptio

In [25]:
fmd = load_json(fm_dance_path)
fmp = load_json(fm_pop_path)
fm_all = list(fmd.values())+list(fmp.values())
arr2csv(fm_all, fm_csv)
df = pd.read_csv(fm_csv); df.head()

,midi,time_signature,title,quarter_length,inferred_key,inferred_offset,source,instruments,genres,artist,seconds,bpm
0,midi_sources/freemidi/genre-dance-eletric/Glor...,NaN,I'm Not Givin' You Up,NaN,NaN,NaN,freemidi,NaN,dance,Gloria Estefan,NaN,NaN
1,midi_sources/freemidi/genre-dance-eletric/Tune...,NaN,Bounce,68.0,F major,-5.0,freemidi,NaN,dance,Tune Up,28.732394,142.0
2,midi_sources/freemidi/genre-dance-eletric/Radi...,4/4,Fitter Happier,108.0,G minor,2.0,freemidi,"Piano,Piano",dance,Radiohead,85.263158,76.0
3,midi_sources/freemidi/genre-dance-eletric/Daft...,4/4,The Grid,68.0,C# minor,-4.0,freemidi,"StringInstrument,StringInstrument,StringInstru...",dance,Daft Punk,40.000000,102.0
4,midi_sources/freemidi/genre-dance-eletric/Bjor...,4/4,Glora,613/6,A major,3.0,freemidi,"Flute,Flute,Flute,Flute",dance,Bjork,NaN,65.0


### Gather Cprato

In [26]:
cp_path, cp_json, cp_csv = create_paths('cprato')
list(cp_path.glob('*'))[:5]

[PosixPath('data/midi/v5/midi_sources/cprato/Basto - Again And Again (midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v5/midi_sources/cprato/The Weeknd ft. Lana Del Rey - Stargirl Interlude  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v5/midi_sources/cprato/Two Steps From Hell - Magic of Love  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v5/midi_sources/cprato/Bermuda Loverz - My Girl (Ladidada) (Rimini Rockaz Radio Edit) (Midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v5/midi_sources/cprato/Cascada - Everytime We Touch (Midi By Carlo Prato) (www.cprato.com).mid')]

In [27]:
def parse_cprato_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1].replace('(midi by Carlo Prato) (www.cprato.com)', '')
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': genres,
        'source': source
    }

In [28]:
cp_meta = partial(parse_cprato_songs, genres='EDM,inferred', source='cprato')
file_list = get_files(cp_path, extensions=['.mid'], recurse=True)
cp_md = parse_midi_dir(file_list, cp_json, 
                       base_path=version_path, meta_func=cp_meta)

In [29]:
cp = load_json(cp_json)
arr2csv(cp.values(), cp_csv)
df = pd.read_csv(cp_csv); df.head()

,midi,time_signature,title,quarter_length,inferred_key,inferred_offset,source,instruments,genres,artist,seconds,bpm
0,midi_sources/cprato/Robert Miles - Fable (mid...,4/4,Fable,68.0,A major,3,cprato,"Piano,Piano,Piano,Piano,Piano,Piano","EDM,inferred",Robert Miles,29.982363,136.08
1,midi_sources/cprato/BT vs. Sasha - Remember Ma...,4/4,Remember Magnetic Nord,132.0,F minor,4,cprato,NaN,"EDM,inferred",BT vs. Sasha,59.104478,134.00
2,midi_sources/cprato/deadmau5 - Cat Thruster (...,4/4,Cat Thruster,68.0,B- minor,-1,cprato,"Fretless Bass,Fretless Bass,Fretless Bass,Pian...","EDM,inferred",deadmau5,35.478261,115.00
3,midi_sources/cprato/GFRIEND - Navillera (midi...,4/4,Navillera,68.0,B minor,-2,cprato,"StringInstrument,StringInstrument,StringInstru...","EDM,inferred",GFRIEND,32.640000,125.00
4,midi_sources/cprato/Oh Wonder - All We Do (mi...,4/4,All We Do,100.0,C# major,-1,cprato,"Flute,Flute,Flute,Piano,Piano,Piano,Fretless B...","EDM,inferred",Oh Wonder,45.801527,131.00


### Gather MidiWorld

In [30]:
mw_path, mw_json, mw_csv = create_paths('midiworld')

In [31]:
def parse_midiworld_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': 'pop,inferred',
        'source': 'midiworld'
    }

In [32]:
file_list = get_files(mw_path/'named_midi', extensions=['.mid'], recurse=True)
mw_md = parse_midi_dir(file_list, mw_json, base_path=version_path, meta_func=parse_midiworld_songs)

Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/Bomfunk_MCs_-_Uprocking_Beats.mid badly formated midi bytes, got: b'RIFF\xb8\xa7\x00\x00RMIDdata\x04\xa7\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/Nirvana_-_Polly.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/Nirvana_-_Pennyroyal_Tea.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/Nirvana_-_Radio_Friendly_Unit_Shifter.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/Bomfunk_MCs_-_B-Boys_And_Fly-Girls.mid badly formated midi bytes, got: b'RIFF^\xd8\x00\x00RMIDdata\xa1\xd7\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/Nightwish_-_Feel_For_You.mid badly formated midi bytes, got: b'RIFFr\x9a\x00\x00RMIDdatae\x9a\x00\x00'
Midi Exeption: data/midi/v5/midi_sources/midiworld/named_midi/B

In [33]:
mw = load_json(mw_json)
arr2csv(mw.values(), mw_csv)
df = pd.read_csv(mw_csv); df.head()

,midi,time_signature,title,quarter_length,inferred_key,inferred_offset,source,instruments,genres,artist,seconds,bpm
0,midi_sources/midiworld/named_midi/Kona_-_Druml...,4/4,Drumloop,3.5,B minor,-2.0,midiworld,Piano,"pop,inferred",Kona,1.193182,176.0
1,midi_sources/midiworld/named_midi/Cameron_Lee_...,4/4,Cue,4.0,D minor,-5.0,midiworld,Piano,"pop,inferred",Cameron Lee Simpson,2.526316,95.0
2,midi_sources/midiworld/named_midi/TV_Themes_-_...,4/4,Millenium,88.75,A major,3.0,midiworld,"Timpani,Taiko,Violin,Contrabass,Voice","pop,inferred",TV Themes,41.929134,127.0
3,midi_sources/midiworld/named_midi/TV_Themes_-_...,4/4,Looney Tunes,154/3,C major,0.0,midiworld,"Marimba,Acoustic Bass,Trombone,StringInstrumen...","pop,inferred",TV Themes,19.250000,160.0
4,midi_sources/midiworld/named_midi/Video_Game_T...,4/4,Diddy Kong,132.0,B minor,-2.0,midiworld,"Fretless Bass,Harmonica","pop,inferred",Video Game Themes,73.333333,108.0


### Gather Wikifonia

In [34]:
wf_path, wf_json, wf_csv = create_paths('wikifonia')

In [35]:
def parse_wikifonia_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'mxl': relative_path(fp),
        'genres': 'pop,inferred',
        'source': 'wikifonia'
    }

In [36]:
# Warning: if you get a deadlock: 
# PosixPath('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid') is broken
file_list = get_files(wf_path, extensions=['.mxl'], recurse=True)
wf_md = parse_midi_dir(file_list, wf_json, base_path=version_path, meta_func=parse_wikifonia_songs)

Midi Exeption: data/midi/v5/midi_sources/wikifonia/Django Reinhardt - Douce Ambiance.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Charlie Parker - Scrapple From The Apple.mxl Degree not in specified chord: 9


musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Midi Exeption: data/midi/v5/midi_sources/wikifonia/Bach ? - Four Part Choralforwikifoniachannel04quant.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v5/midi_sources/wikifonia/blah3 - blah.mxl failed to get likely keys for Stream component
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Monty Python - Always look on the bright side of kive.mxl got a negative delta time
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Fraser Smith, Adele Adkins - Set Fire To The Rain.mxl float division by zero
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Spencer, James F. Hanley - Mary Is Talking.mxl failed to get likely keys for Stream component
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Maurice Williams - Stay.mxl unsupported operand type(s) for /: 'int' and 'NoneType'
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Gigi Gryce, Jon Hendricks - Social Call.mxl unsupported operand type(s) for /: 'int' and 'NoneType'
Midi Exeption: data/midi

musicxml.xmlToM21: WARNING: Line <bracket> stop without start


Midi Exeption: data/midi/v5/midi_sources/wikifonia/Joe Harnell - The Lonely Man.mxl Degree not in specified chord: 11
Midi Exeption: data/midi/v5/midi_sources/wikifonia/L. van Beethoven - Himno a la Alegria.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v5/midi_sources/wikifonia/ARR. by Peter Sweeney - WONDERFUL TONIGHT.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Dot145 - Precession of the Equinoxes.mxl Degree not in specified chord: 9
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Ray Henderson, Joseph Widow Young, Sam M. Lewis - Five Foot Two, Eyes Of Blue .mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Clifford Brown - Daahoud.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v5/midi_sources/wikifonia/Gerry Goffin, Michael Masser - Nothing's Gonna Change My Love 

In [37]:
wf = load_json(wf_json)
arr2csv(wf.values(), wf_csv)
df = pd.read_csv(wf_csv); df.head()

,midi,time_signature,title,quarter_length,inferred_key,inferred_offset,source,instruments,genres,artist,mxl,seconds,bpm
0,NaN,NaN,Douce Ambiance,NaN,NaN,NaN,wikifonia,NaN,"pop,inferred",Django Reinhardt,midi_sources/wikifonia/Django Reinhardt - Douc...,NaN,NaN
1,midi_sources/from_mxl/wikifonia/Hungarian folk...,3/4,"Hej, Varg?n? k?poszt?t f?z",24.0,D minor,-5.0,wikifonia,Voice,"pop,inferred",Hungarian folk song,midi_sources/wikifonia/Hungarian folk song - H...,NaN,NaN
2,NaN,NaN,Scrapple From The Apple,NaN,NaN,NaN,wikifonia,NaN,"pop,inferred",Charlie Parker,midi_sources/wikifonia/Charlie Parker - Scrapp...,NaN,NaN
3,midi_sources/from_mxl/wikifonia/Joe Henderson ...,4/4,Mamacita,64.0,F minor,4.0,wikifonia,NaN,"pop,inferred",Joe Henderson,midi_sources/wikifonia/Joe Henderson - Mamacit...,NaN,NaN
4,midi_sources/from_mxl/wikifonia/Hungarian folk...,4/4,?n az ?jjel nem aludtam egy ?r?t,48.0,C major,0.0,wikifonia,Grand Piano,"pop,inferred",Hungarian folk song,midi_sources/wikifonia/Hungarian folk song - ?...,NaN,NaN


### Yamaha - piano

In [38]:
ec_path, ec_json, ec_csv = create_paths('ecomp')
ec_song = ec_path/'song_list.json'
list(ec_path.glob('*'))[:5]

[PosixPath('data/midi/v5/midi_sources/ecomp/song_list.json'),
 PosixPath('data/midi/v5/midi_sources/ecomp/2017'),
 PosixPath('data/midi/v5/midi_sources/ecomp/2008'),
 PosixPath('data/midi/v5/midi_sources/ecomp/2006'),
 PosixPath('data/midi/v5/midi_sources/ecomp/2004')]

In [39]:
ec_songs = load_json(ec_path/'song_list.json')

In [40]:
def parse_ecomp_songs(fp):
    song_info = ec_songs[fp.stem]
    return {
        'artist': song_info['artist'],
        'title': song_info['title'],
        'mxl': relative_path(fp),
        'genres': 'classical',
        'source': 'ecomp'
    }

In [41]:
file_list = get_files(ec_path, extensions=['.mxl'], recurse=True)

In [42]:
sorted_files = sorted([fp.stat().st_size/1000 for fp in file_list])[-200:]

In [43]:
file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 400] # over 200mb takes crazy long to analyze
ec_md = parse_midi_dir(file_list, ec_json, base_path=version_path, meta_func=parse_ecomp_songs)

Midi Exeption: data/midi/v5/midi_sources/ecomp/2017/SunY03.mxl got a negative delta time
Midi Exeption: data/midi/v5/midi_sources/ecomp/2006/Tysman02.mxl got a negative delta time
Midi Exeption: data/midi/v5/midi_sources/ecomp/2004/BLINOV01.mxl got a negative delta time
Midi Exeption: data/midi/v5/midi_sources/ecomp/2009/Na03.mxl got a negative delta time
Midi Exeption: data/midi/v5/midi_sources/ecomp/2002/sun01.mxl got a negative delta time


In [44]:
ec = load_json(ec_json)
arr2csv(ec.values(), ec_csv)
df = pd.read_csv(ec_csv); df.head()

,midi,time_signature,title,quarter_length,inferred_key,inferred_offset,source,instruments,genres,artist,mxl,seconds,bpm
0,midi_sources/from_mxl/ecomp/2017/WangH09.mid,3/4,II. Intermezzo in A Minor,375.0,A minor,0,ecomp,"Piano,Piano",classical,,midi_sources/ecomp/2017/WangH09.mxl,212.264151,106.0
1,midi_sources/from_mxl/ecomp/2017/KaiRuiR06.mid,4/4,Etude Op. 10 No. 2 in A Minor,212.0,A minor,0,ecomp,"Piano,Piano",classical,Frédéric Chopin,midi_sources/ecomp/2017/KaiRuiR06.mxl,79.500000,160.0
2,midi_sources/from_mxl/ecomp/2017/ZhangE05.mid,4/4,"Sonata in F Minor, K. 466",37441/120,F minor,4,ecomp,"Piano,Piano",classical,Giuseppe Scarlatti,midi_sources/ecomp/2017/ZhangE05.mxl,236.968354,79.0
3,midi_sources/from_mxl/ecomp/2017/TongB01.mid,4/4,"Prelude and Fugue in G-sharp Minor, WTC I, ...",443707/840,G# minor,1,ecomp,"Piano,Piano",classical,Johann Sebastian Bach,midi_sources/ecomp/2017/TongB01.mxl,268.587772,118.0
4,midi_sources/from_mxl/ecomp/2017/WangA01.mid,4/4,"Prelude and Fugue in E Major, WTC I, BWV 854",184.0,E major,-4,ecomp,"Piano,Piano",classical,Johann Sebastian Bach,midi_sources/ecomp/2017/WangA01.mxl,149.189189,74.0


### Reformatting code

In [45]:
# def reformat_json(out_path):
#     f2m = load_json(out_path)
#     m = {relative_path(k):v for k,v in f2m.items()}
#     save_json(m, out_path)
#     return m

In [46]:
# def reformat_key(k):
#     return k.replace('data/midi/midi_sources_fromxml_v3','midi_sources/from_mxl')

In [47]:
# def reformat_data(d):
#     if 'midi' in d: d['midi'] = reformat_key(d['midi'])
#     if 'mxl' in d: d['mxl'] = d['mxl'].replace('data/midi/midi_sources_v3', 'midi_sources')
#     return d

In [48]:
# f2m = load_json(clc_json)
# m = {reformat_data(v)['mxl']:reformat_data(v) for k,v in f2m.items() if 'mxl' in reformat_data(v)}
# save_json(m, clc_json)

### Classic Piano

In [49]:
clc_path, clc_json, clc_csv = create_paths('classic_piano')
list(clc_path.glob('*'))[:5]

[PosixPath('data/midi/v5/midi_sources/classic_piano/liz_rhap15_format0.mxl'),
 PosixPath('data/midi/v5/midi_sources/classic_piano/ty_september_format0.mxl'),
 PosixPath('data/midi/v5/midi_sources/classic_piano/clementi_opus36_2_2_format0.mid'),
 PosixPath('data/midi/v5/midi_sources/classic_piano/mz_333_2_format0.mid'),
 PosixPath('data/midi/v5/midi_sources/classic_piano/haydn_7_1_format0.mid')]

In [50]:
def parse_classic_songs(fp):
    name = fp.with_suffix('').name.split('_')
    artist = name[0]
    title = ' '.join(name[1:])
    return {
        'artist': artist,
        'title': title,
        'mxl': relative_path(fp),
        'genres': 'classical',
        'source': 'classical_piano'
    }

In [51]:
file_list = get_files(clc_path, extensions=['.mxl'], recurse=True)

In [52]:
file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 350] # over 200mb takes crazy long to analyze

In [53]:
clc_md = parse_midi_dir(file_list, clc_json, base_path=version_path, meta_func=parse_classic_songs)

In [54]:
clc = load_json(clc_json)
arr2csv(clc.values(), clc_csv)
df = pd.read_csv(clc_csv); df.head()

,midi,time_signature,title,quarter_length,inferred_key,inferred_offset,source,instruments,genres,artist,mxl,seconds,bpm
0,midi_sources/from_mxl/classic_piano/chpn-p11_f...,6/8,format0,81.0,B major,1,classical_piano,"Piano,Piano",classical,chpn-p11,midi_sources/classic_piano/chpn-p11_format0.mxl,29.635549,111.0
1,midi_sources/from_mxl/classic_piano/burg_gesch...,6/8,geschwindigkeit format0,141.0,C major,0,classical_piano,"Piano,Piano",classical,burg,midi_sources/classic_piano/burg_geschwindigkei...,68.712982,125.0
2,midi_sources/from_mxl/classic_piano/haydn_8_2_...,3/4,8 2 format0,96.0,G major,5,classical_piano,"Piano,Piano",classical,haydn,midi_sources/classic_piano/haydn_8_2_format0.mxl,57.682776,100.0
3,midi_sources/from_mxl/classic_piano/scn15_11_f...,2/4,11 format0,128.0,G major,5,classical_piano,"Piano,Piano",classical,scn15,midi_sources/classic_piano/scn15_11_format0.mxl,76.424015,97.0
4,midi_sources/from_mxl/classic_piano/scn15_7_fo...,4/4,7 format0,132.0,F major,-5,classical_piano,"Piano,Piano",classical,scn15,midi_sources/classic_piano/scn15_7_format0.mxl,128.220469,100.0


### Creating CSV

In [55]:
all_csvs = [create_paths(s)[-1] for s in sources]
all_dfs = [pd.read_csv(csv) for csv in all_csvs]

In [56]:
merged_df = pd.concat(all_dfs, sort=False)
merged_df = merged_df.reset_index(drop=True); merged_df

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,midi,inferred_key,inferred_offset,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36,1.0,hooktheory,wayne-sharpe,C,...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,0.0,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32,1.0,hooktheory,wayne-sharpe,C,...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,0.0,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16,6.0,hooktheory,what-a-day,D,...,midi_sources/hooktheory/pianoroll/w/what-a-day...,A minor,0.0,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32,6.0,hooktheory,whiteflame,D,...,midi_sources/hooktheory/pianoroll/w/whiteflame...,D minor,-5.0,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32,1.0,hooktheory,wham,Db,...,midi_sources/hooktheory/pianoroll/w/wham/last-...,C# major,-1.0,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN
5,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32,6.0,hooktheory,whiteflame,D,...,midi_sources/hooktheory/pianoroll/w/whiteflame...,D minor,-5.0,"J-Pop,Pop",pre-chorus,"verse,pre-chorus,chorus",4.0,152.0,senbonzakura - pre-Pre-Chorus,NaN
6,https://www.hooktheory.com/theorytab/view/will...,Piano,-4.0,4/4,on-the-road-again,28,1.0,hooktheory,willie-nelson,E,...,midi_sources/hooktheory/pianoroll/w/willie-nel...,A major,3.0,NaN,bridge,"verse,bridge",4.0,112.0,On The Road Again,NaN
7,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32,1.0,hooktheory,wham,Db,...,midi_sources/hooktheory/pianoroll/w/wham/last-...,B- minor,-1.0,Holiday,chorus,"intro,verse,chorus",4.0,108.0,Last Christmas Chorus,NaN
8,https://www.hooktheory.com/theorytab/view/will...,Piano,-4.0,4/4,on-the-road-again,28,1.0,hooktheory,willie-nelson,E,...,midi_sources/hooktheory/pianoroll/w/willie-nel...,E major,-4.0,NaN,verse,"verse,bridge",4.0,112.0,On The Road Again,NaN
9,https://www.hooktheory.com/theorytab/view/weez...,NaN,0.0,NaN,beverly-hills,NaN,NaN,hooktheory,weezer,C,...,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,NaN,intro-and-verse,intro-and-verse,4.0,128.0,My New Song,NaN


In [57]:
[df.shape for df in all_dfs], merged_df.shape

([(19876, 21),
  (5784, 12),
  (4711, 12),
  (2538, 13),
  (314, 12),
  (328, 13),
  (6391, 13)],
 (39942, 22))

In [58]:
merged_df.to_csv(all_csv, index=False)